In [14]:
import wikisearch
import wikipedia
import string
import sys
import numpy as np

In [6]:
def check_links_subjects(model,current,subs,visited):
    """Uses the input model to check the similarity of the links in the current page
    to the title of the target page."""
    #get links from current
    links = current.links
    success = []
    errors = []
    
    #check links against model for relevence to target subject
    for l in links:
        for word in l.split(' '):
            word = word.lower()
            try:
                for targetword in subs:
                    dist = model.wv.distance(word,targetword.lower())
                    if dist < 0.08 and l not in visited:
                        success.append((l,dist))
                        break
            except KeyError:
                errors.append(word)
    #if related links found, use most related link, otherwise random
    if len(success) > 0:
        success.sort(key=lambda tup: tup[1])
        return success[0][0]
    else:
        skiplist = ['Wikipedia', 'Category']
        title = links[np.random.randint(0,len(links))]
        while any(sub in title for sub in skiplist):
            title = links[np.random.randint(0,len(links))]
        return title

In [15]:
def search_subject(start,subs):
    start_page = wikipedia.page(start)
    closest = ("none",sys.maxsize)
    print("0 : " + wikipedia.page(start).title)
    # if 'path' in locals():
    #     if target_page not in path:
    path = [start_page]
    visited = set()
    visited.add(start)
    title = start
    exit = False
    #i = 0

    for i in range(25):
    #while not exit:
        try:

            #get next page
            current = wikipedia.page(title)
            #test for target page
            #if current not in path:
            path.append(current)
            #create model
            model = wikisearch.make_model_new(path)
            #get common words in current page
            top_20 = wikisearch.get_50_most_common(current.content)[:20]

            #test for similarity to subject
            for word,freq in top_20:
                match = []
                try:
                    for sub in subs:
                        dist = model.wv.distance(word,sub)
                        if dist < 0.06:
                            match.append(True)
                        else:
                            match.append(False)
                except KeyError:
                    pass
            if len(match) > 0 and False not in match:
                break
            search_success = True
        except wikipedia.exceptions.DisambiguationError:
            search_success = False
        except wikipedia.exceptions.PageError:
            search_success = False

        #get next link
        title = check_links_subjects(model,current,subs,visited)
        visited.add(title)
        if i % 10 == 0:
            print(".",end="")
        #i += 1

    print("Page Found!") if exit else print("Not Found")
    print(str(i) + ": Matching page: " + title)

In [ ]:
search_subject('Wonder Woman',['woman','superhero'])

0 : Wonder Woman
..

In [2]:
ww = wikipedia.page('Wonder Woman')
model = wikisearch.make_model_new([ww])

In [3]:
len(model.wv.vocab)

1389

In [28]:
text = ww.content.split(' ')

In [16]:
wikisearch.get_50_most_common(ww.content)

[('woman', 300),
 ('wonder', 294),
 ('diana', 162),
 ('comic', 64),
 ('power', 60),
 ('charact', 57),
 ('amazon', 56),
 ('marston', 49),
 ('hippolyta', 40),
 ('women', 40),
 ('origin', 39),
 ('world', 39),
 ('stori', 34),
 ('would', 34),
 ('also', 32),
 ('issu', 32),
 ('first', 30),
 ('make', 30),
 ('superman', 29),
 ('steve', 29),
 ('princ', 28),
 ('time', 27),
 ('outfit', 27),
 ('bracelet', 26),
 ('earth', 26),
 ('continu', 25),
 ('golden', 25),
 ('seri', 25),
 ('later', 25),
 ('take', 24),
 ('role', 24),
 ('creat', 23),
 ('year', 23),
 ('lasso', 23),
 ('come', 23),
 ('includ', 22),
 ('know', 22),
 ('strength', 22),
 ('kill', 22),
 ('book', 21),
 ('themyscira', 21),
 ('give', 21),
 ('return', 21),
 ('costum', 21),
 ('reveal', 21),
 ('abil', 21),
 ('like', 21),
 ('island', 20),
 ('god', 20),
 ('zeus', 20)]

12

In [ ]:
model.wv.distance('diana','woman')